In [1]:
import numpy as np
from bitarray import bitarray
from scipy.io import wavfile
import matplotlib.pyplot as plt
from IPython.display import Audio

In [2]:
def compute_msg_bits(msg_file, cover_aud_file, num_segments):
    '''
    Tính message bits (đã có đuôi 100...) từ message file.
    Các tham số:
        msg_file (str): Tên file chứa secret message.
        cover_aud_file (str): Tên file chứa cover audio.
        num_segments (int): Số lượng đoạn sóng dùng để nhúng (mỗi đoạn sẽ nhúng một bit).
    Giá trị trả về:
        Mảng numpy (một chiều, dtype = bool): message bits (đã thêm 100...), 
                                              chiều dài của mảng này = num_segments,
                                              nếu không đủ chỗ nhúng thì trả về mảng rỗng.
    '''
    # Đọc msg file
    with open(msg_file, 'r') as f:
        msg = f.read()
    
    # Chuyển msg thành msg bits
    msg_bits = bitarray(); msg_bits.fromstring(msg)
   
    # Kiểm xem có nhúng được không?
    _, cover_samples = wavfile.read(cover_aud_file)
    if len(msg_bits) + 1 > num_segments:
        print('Cannot embed!')
        return np.array([])  
    
    # Thêm '100...' vào msg bits
    msg_bits.extend('1' + '0' * (num_segments - len(msg_bits) - 1))
    
    return np.array(list(msg_bits))

In [3]:
cover_aud_file = 'stego.wav'
num_segments = 64
msg_bits = compute_msg_bits('msg.txt', 'cover.wav', num_segments)
time_delta0 = 0.03
time_delta1 = 0.031
decay_rate = 1
stego_aud_file = 'stego.wav'

In [4]:
# Đọc file âm thanh phủ - cover_aud_file
rate, cover_samples = wavfile.read(cover_aud_file)
len_cover_samples = len(cover_samples)

# Xác định các thông số
len_sample_segment = int(len(cover_samples)/ num_segments)    # Số lượng samples trên mỗi đoạn được nhúng bit - tạo echo  
len_echo = num_segments * len_sample_segment                  # Số lượng samples nhúng được bit - tạo echo (Kích thước của echo) (= 279488)
len_not_echo = len_cover_samples - len_echo                   # Số lượng samples không được nhúng bit - tạo echo (=22)

sample_not_echo = cover_samples[len_echo:]                    # Danh sách các samples không được nhúng bit - tạo echo    
# ------------------------------------------------------------------------------------------------------------------------------

In [5]:
# Tạo echo ứng với bit 0
echo_delay_bit0 = np.zeros_like(cover_samples)                                                 # Khởi tạo echo
rate_delay_bit0 = int(time_delta0 * rate)                                                      # Khoảng trễ của echo bit 0
echo_delay_bit0[rate_delay_bit0:] = cover_samples[:-rate_delay_bit0] * decay_rate              # Tạo echo chứa bit 0 với khoảng trễ rate_delay_bit0
echo_bit0 = echo_delay_bit0[0:len_echo]                                                        # Cắt bỏ đoạn samples tạo dư (Đoạn tạo độ trễ có kích thước 279510 sample mà chỉ 279488 sample cần tạo echo)
echo_bit0 = np.append(echo_bit0, sample_not_echo)                                              # Bổ sung thêm đoạn samples không có echo    
# ------------------------------------------------------------------------------------------------------------------------------  

In [6]:
# Tạo echo ứng với bit 1
echo_delay_bit1 = np.zeros_like(cover_samples)                                                 # Khởi tạo echo
rate_delay_bit1 = int(time_delta1 * rate)                                                      # Khoảng trễ của echo bit 1
echo_delay_bit1[rate_delay_bit1:] = cover_samples[:-rate_delay_bit1] * decay_rate              # Tạo echo chứa bit 1 với khoảng trễ rate_delay_bit1
echo_bit1 = echo_delay_bit1[0:len_echo]                                                        # Cắt bỏ đoạn samples tạo dư (Đoạn tạo độ trễ có kích thước 279510 sample mà chỉ 279488 sample cần tạo echo)
echo_bit1 = np.append(echo_bit1, sample_not_echo)                                              # Bổ sung thêm đoạn samples không có echo    
# ------------------------------------------------------------------------------------------------------------------------------

In [7]:
# Tạo mixer1 dựa vào chuỗi bit nhúng
idx_Mixer1 = 0
mixer1 = np.empty_like(cover_samples)
for idx_Segments in range(0, num_segments): # Xét trên mỗi đoạn sample gán bit tương ứng trên mỗi đoạn
    mixer1[idx_Mixer1: idx_Mixer1 + len_sample_segment] = int(msg_bits[idx_Segments])
    idx_Mixer1 += len_sample_segment

# Tạo mixer0 dựa mixer1
mixer0 = 1 - mixer1

In [8]:
# Tạo stego_audio
cover_audio_bit0 = cover_samples + echo_bit0                                 # Tạo đoạn echo chứa bit 0 để ẩn tin mật
cover_audio_bit1 = cover_samples + echo_bit1                                 # Tạo đoạn echo chứa bit 1 để ẩn tin mật

In [11]:
print(len(cover_audio_bit0))
print(len(cover_audio_bit1))

279510
279510


In [12]:
stego_audio = (cover_audio_bit0 * mixer0) + (cover_audio_bit1 * mixer1)
print(len(stego_audio))

279510


In [13]:
stego_audio2 = stego_audio[0: len_echo]
print(len(stego_audio2))

279488


In [14]:
stego_audio3 = np.append(stego_audio2, sample_not_echo)
print(len(stego_audio3))

279510


In [15]:
np.array_equal(stego_audio,stego_audio3)

False

In [18]:
print(sample_not_echo)

[-3  5 -4  0  0 -2  1 -1  1 -1  2 -2  0  2 -3  5 -2  2  2 -3  4 -3]


In [16]:
print(stego_audio[279488:])

[-6 10 -8  0  0 -4  2 -2  2 -2  4 -4  0  4 -6 10 -4  4  4 -6  8 -6]


In [17]:
print(stego_audio3[279488:])

[-3  5 -4  0  0 -2  1 -1  1 -1  2 -2  0  2 -3  5 -2  2  2 -3  4 -3]


In [ ]:
#wavfile.write(stego_aud_file, rate, stego_audio)